In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid # A grid that allows multiple agents to share a cell

import matplotlib.pyplot as plt

In [ ]:
class Dragon(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.weight = 10
        self.energy = 50
        self.dead = False
        
    def migrate(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        
    def damage_environment(self):
        x, y = self.pos
        damage = min(self.model.trees[x][y], DAMAGE_PER_WEIGHT * self.weight)
        self.model.trees[x][y] -= damage
        
    def step(self):
        if self.dead:
            return
        
        self.energy -= DAILY_COST
        x, y = self.pos
        
        if (self.energy < HUNTING_THRESHOLD):
            if self.model.deers[x][y] < MIGRATION_THRESHOLD:
                self.migrate()
            else:
                hunted = min(self.model.deers[x][y], HUNTING_EFFECTIVENESS)
                self.energy += hunted * GAIN_FROM_FOOD
                self.model.deers[x][y] -= hunted
        if self.energy > GROWTH_THRESHOLD:
            self.weight += 1
            self.energy -= GROWTH_COST
        if self.energy < SHRINK_THRESHOLD:
            self.weight -= 1
            
        if self.weight <= 0:
            self.dead = True
            
        self.damage_environment()
        
class DragonEcoModel(Model):
    def __init__(self, N, width, height):
        self.N = N
        self.grid = MultiGrid(width, height, False)
        self.schedule = RandomActivation(self)
        
        self.deers = [ [ self.random.gauss(INIT_DEERS, INIT_DEERS / 10)
                for _ in range(height)] for _ in range(width)]
         
        self.trees = [ [ self.random.gauss(INIT_TREES, INIT_TREES / 10)
                for _ in range(height)] for _ in range(width)]
        
        for i in range(self.N):
            d = Dragon(i, self)
            self.schedule.add(d)
            
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            
            self.grid.place_agent(d, (x, y))
            
    def step(self):
        self.schedule.step()
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                self.deers[x][y] += DEER_GROWTH
                self.trees[x][y] += TREE_GROWTH
                
                self.deers[x][y] = min(self.deers[x][y], DEER_CAP)
                self.trees[x][y] = min(self.trees[x][y], TREE_CAP)

In [ ]:
INIT_WEIGHT = 10
INIT_ENERGY = 30
GAIN_FROM_FOOD = 5
HUNTING_THRESHOLD = 50
GROWTH_THRESHOLD = 70
GROWTH_COST = 20
SHRINK_THRESHOLD = 30
DAILY_COST = 30
MIGRATION_THRESHOLD = 20

DAMAGE_PER_WEIGHT = 10
HUNTING_EFFECTIVENESS = 70

INIT_DEERS = 100
INIT_TREES = 100

DEER_GROWTH = 1
TREE_GROWTH = 0.5

DEER_CAP = 100
TREE_CAP =100

N_DRAGONS = 3
WIDTH = 10
HEIGHT = 10

In [ ]:
model = DragonEcoModel(N_DRAGONS, WIDTH, HEIGHT)

avg_dragon_weights = []
avg_dragon_energies = []
alive_dragons = []

for i in range(TOTAL_TIME):
    if i % 10 == 0:
        print("Step " + str(i))
        
        print("Deer Distribution")
        plt.imshow(model.deers, interpolation="nearest")
        plt.colorbar()
        plt.show()
        
        print("Tree Distribution")
        plt.imshow(model.trees, interpolation="nearest")
        plt.colorbar()
        plt.show()
    
    model.step()
    
    dragon_weights = [d.weight for d in model.schedule.agents if not d.dead]
    dragon_energies = [d.energy for d in model.schedule.agents if not d.dead]
    if len(dragon_weights) > 0:
        avg_dragon_weights.append(sum(dragon_weights) / len(dragon_weights))
    else:
        avg_dragon_weights.append(0)
        
    if len(dragon_energies) > 0:
        avg_dragon_energies.append(sum(dragon_energies) / len(dragon_energies))
    else:
        avg_dragon_energies.append(0)
        
    alive_dragons.append(sum([1 for d in model.schedule.agents if not d.dead]))
    
plt.plot(avg_dragon_energies)
plt.title("Average Dragon Energies")
plt.show()
plt.plot(avg_dragon_weights)
plt.title("Average Dragon Weights")
plt.show()
plt.title("Number of Dragons")
plt.plot(alive_dragons)
plt.show()
    